首先，迁移学习是指，我们应用了 别的已经训练好的完善的模型，然后修改模型的最后一层(或者是更多)，就是迁移学习

In [2]:
import os
import math
import random
import shutil
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sn
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

In [3]:
vgg_model = tf.keras.applications.vgg16.VGG16()
print(type(vgg_model))
vgg_model.summary()

<class 'keras.engine.functional.Functional'>
Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (Ma

In [5]:
model = keras.models.Sequential()
for layer in vgg_model.layers[0:-1]:#这里我们去掉了最后一层
    model.add(layer)

In [6]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0         
                                                                 
 block3_conv1 (Conv2D)       (None, 56, 56, 256)      

目前我们去掉了最后的outinputs，现在对于原先的网络层，我们不需要进行训练了，我们只需要训练最后修剪的部分，所以我们将去除最后一层的model进行训练去除(就是不可训)

In [7]:
for layer in model.layers:
    layer.trainable = False
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0         
                                                                 
 block3_conv1 (Conv2D)       (None, 56, 56, 256)      

现在我们就能看到summary里面的Trainable params和Non-trainable params的参数变化。

然后我们再加入我们需要训练的dense layer

In [8]:
model.add(layers.Dense(5))

In [9]:
loss = keras.losses.SparseCategoricalCrossentropy(from_logits=True)
optim=keras.optimizers.Adam(learning_rate=0.001)
metrics = ["accuracy"]
model.compile(optimizer=optim,loss=loss,metrics=metrics)

In [10]:
preprocess_input = tf.keras.applications.vgg16.preprocess_input

In [12]:
names = ["YODA","LUKE SKYWALKER","R2-D2","MACE WINDU","GENERAL GRIEVOUS"]

In [13]:
train_gen = keras.preprocessing.image.ImageDataGenerator(preprocessing_function=preprocess_input)
valid_gen = keras.preprocessing.image.ImageDataGenerator(preprocessing_function=preprocess_input)
test_gen = keras.preprocessing.image.ImageDataGenerator(preprocessing_function=preprocess_input)

train_batches = train_gen.flow_from_directory(
    'C:/Users/zzy/DL_dataset/lego/star-wars-images/train',
    target_size=(224,224),
    class_mode='sparse',
    batch_size=4,
    shuffle=True,
    color_mode="rgb",
    classes=names
)

val_batches = valid_gen.flow_from_directory(
    'C:/Users/zzy/DL_dataset/lego/star-wars-images/val',
    target_size=(224,224),
    class_mode='sparse',
    batch_size=4,
    shuffle=False,
    color_mode="rgb",
    classes=names
)

test_batches = valid_gen.flow_from_directory(
    'C:/Users/zzy/DL_dataset/lego/star-wars-images/test',
    target_size=(224,224),
    class_mode='sparse',
    batch_size=4,
    shuffle=False,
    color_mode="rgb",
    classes=names
)

Found 40 images belonging to 5 classes.
Found 19 images belonging to 5 classes.
Found 4 images belonging to 5 classes.


In [14]:
epochs=30
#keras回调
early_stopping= keras.callbacks.EarlyStopping(
    monitor="val_loss",
    patience=5,
    verbose=2
)
history = model.fit(
                    train_batches,
                    validation_data=val_batches,
                     callbacks=[early_stopping],
                    epochs=epochs,
                    verbose=2)

Epoch 1/30
10/10 - 5s - loss: 1.9051 - accuracy: 0.4250 - val_loss: 0.8304 - val_accuracy: 0.7368 - 5s/epoch - 525ms/step
Epoch 2/30
10/10 - 4s - loss: 0.2261 - accuracy: 0.9000 - val_loss: 0.3892 - val_accuracy: 0.8947 - 4s/epoch - 417ms/step
Epoch 3/30
10/10 - 4s - loss: 0.0242 - accuracy: 1.0000 - val_loss: 0.3338 - val_accuracy: 0.8947 - 4s/epoch - 399ms/step
Epoch 4/30
10/10 - 4s - loss: 0.0132 - accuracy: 1.0000 - val_loss: 0.3326 - val_accuracy: 0.8947 - 4s/epoch - 395ms/step
Epoch 5/30
10/10 - 4s - loss: 0.0077 - accuracy: 1.0000 - val_loss: 0.3115 - val_accuracy: 0.8947 - 4s/epoch - 395ms/step
Epoch 6/30
10/10 - 4s - loss: 0.0038 - accuracy: 1.0000 - val_loss: 0.2990 - val_accuracy: 0.8947 - 4s/epoch - 403ms/step
Epoch 7/30
10/10 - 4s - loss: 0.0026 - accuracy: 1.0000 - val_loss: 0.2941 - val_accuracy: 0.8947 - 4s/epoch - 395ms/step
Epoch 8/30
10/10 - 4s - loss: 0.0021 - accuracy: 1.0000 - val_loss: 0.2928 - val_accuracy: 0.8947 - 4s/epoch - 396ms/step
Epoch 9/30
10/10 - 4s - 

In [15]:
model.evaluate(test_batches,verbose=2)

1/1 - 0s - loss: 6.1090e-04 - accuracy: 1.0000 - 326ms/epoch - 326ms/step


[0.0006108965608291328, 1.0]